In [1]:

# Install required packages
!pip install transformers==4.29.1 tqdm==4.63.2 pandas==1.4.4 wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=0f69ab462999f6620be9737e56a0a6562f14aafcc43b53924b4f5d88417739b6
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d8

In [2]:
cd /content

/content


In [3]:

# Clone the repository for geotagging with BYT5
!git clone https://github.com/Yachay-AI/byt5-geotagging


# Install gdown for Google Drive downloads
!pip install gdown

# Download the dataset
!gdown https://drive.google.com/u/2/uc?id=1thkE-hgT3sDtZqILZH17Hyayy0hkk_jh&export=download

# Unzip the downloaded dataset
!tar xvf challenge_1.tar.gz > /dev/null

# Read sample data
!head data_sample_lc/c_46.json


Cloning into 'byt5-geotagging'...
remote: Enumerating objects: 244, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 244 (delta 54), reused 18 (delta 10), pack-reused 146
Receiving objects: 100% (244/244), 13.03 MiB | 19.49 MiB/s, done.
Resolving deltas: 100% (100/100), done.
Downloading...
From: https://drive.google.com/u/2/uc?id=1thkE-hgT3sDtZqILZH17Hyayy0hkk_jh
To: /content/challenge_1.tar.gz
100% 107M/107M [00:02<00:00, 45.9MB/s] 
{"text": "le estará afectando la menopáusia? https://t.co/ij2y1j53c0", "coordinates": ["-78.77473262476528", "-2.199325289082757"]}
{"text": "acaba de publicar una foto en conocoto, pichincha, ecuador https://t.co/qwrmyfex0a", "coordinates": ["-78.1677027989582", "-0.5747879967862987"]}
{"text": "acaba de publicar una foto en volcán cotacachi https://t.co/vbcw4bfpgn", "coordinates": ["-78.70883119538775", "0.5082920403995214"]}
{"text": "@karlosivan3 sonó gay eso", "coordinates": ["-80.9448

In [4]:
# Import necessary modules
import os
import pandas as pd
import pickle
import torch
from transformers import AutoTokenizer

# Initialize an empty list to hold dataframes
df_list = []

# Loop through each JSON file and append it to df_list
for fn in os.listdir("data_sample_lc"):
  df_list.append(pd.read_json(f"data_sample_lc/{fn}", lines=True))

# Concatenate all dataframes in df_list
df = pd.concat(df_list)

# Extract latitude and longitude from 'coordinates' column
df['lat'] = [x[1] for x in df['coordinates']]
df['lon'] = [x[0] for x in df['coordinates']]

# Drop the 'coordinates' column
df.drop('coordinates', axis=1, inplace=True)

# Shuffle the dataset
df = df.sample(frac=1.0)

# Split dataset into training and testing sets
df.iloc[:len(df)*9//10].to_csv('train.csv')
df.iloc[len(df)*9//10:].to_csv('test.csv')

# Count the number of lines in train.csv and test.csv
!wc -l train.csv
!wc -l test.csv

731994 train.csv
81623 test.csv


In [5]:

# Load cluster data
cluster_df = pd.read_csv('byt5-geotagging/cluster_df.csv')

# Save the clustering model
with open('clustering.pkl', 'wb') as fout:
  pickle.dump((cluster_df, []), fout)


In [ ]:

# Run the training script
# The parameters here are chosen to show a small training run on a small subset of data
!python byt5-geotagging/train_model.py --train_input_file train.csv --test_input_file test.csv --do_train true --do_test true --load_clustering ./ --device cuda --batch_size 64 --keep_layer_count 4 --max_train 96000 --max_test 640


INFO:root:start
start
INFO:root:finish reading test file
finish reading test file
Warning, dropping 1 NaN rows
INFO:root:finish reading train file
finish reading train file
INFO:root:Index(['Unnamed: 0', 'text', 'lat', 'lon', 'coordinates'], dtype='object')
Index(['Unnamed: 0', 'text', 'lat', 'lon', 'coordinates'], dtype='object')
Some weights of the model checkpoint at google/byt5-small were not used when initializing T5EncoderModel: ['decoder.block.1.layer.0.layer_norm.weight', 'decoder.block.2.layer.2.DenseReluDense.wi_1.weight', 'decoder.block.2.layer.2.DenseReluDense.wo.weight', 'lm_head.weight', 'decoder.block.2.layer.0.SelfAttention.v.weight', 'decoder.block.1.layer.1.EncDecAttention.q.weight', 'decoder.block.3.layer.0.SelfAttention.o.weight', 'decoder.block.3.layer.0.layer_norm.weight', 'decoder.block.0.layer.1.EncDecAttention.q.weight', 'decoder.block.3.layer.2.DenseReluDense.wi_1.weight', 'decoder.block.2.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.0.SelfAttention

In [ ]:

# Run the testing script
# For demo purpose, a small subset of validation set is used
!python byt5-geotagging/train_model.py --train_input_file train.csv --test_input_file test.csv --do_test true --load_clustering ./ --load_model_dir models/byt5-class-0 --device cuda --batch_size 32  --max_train 96000 --max_test 640


In [ ]:
cd byt5-geotagging

In [ ]:

# Load trained model and tokenizer
device = 'cuda'
byt5 = torch.load('../models/byt5-class-0')
byt5_tokenizer = AutoTokenizer.from_pretrained('google/byt5-small')

# Make a prediction
text = 'I live in New York'
inputs = byt5_tokenizer(text, return_tensors='pt')['input_ids'].unsqueeze(0)
logits = byt5.to(device)(inputs.to(device))
predicted_cluster = logits.argmax()
confidence = torch.nn.functional.softmax(logits, dim=-1).max().item()
predicted_location = cluster_df.iloc[predicted_cluster.item()]

# Output predicted location and confidence
print(predicted_location['lat'], predicted_location['lng'], confidence)